In [1]:
import requests
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, LineString, Polygon
import folium

Use API to fetch data

In [2]:
# Define the API endpoint
url = "https://www.federalregister.gov/api/v1/documents.json"

# Define query parameters
params = {
    "conditions[publication_date][gte]": "2024-01-01",  # Start date
    "conditions[publication_date][lte]": "2024-12-31",  # End date
    "conditions[agencies][]": "indian-affairs-bureau",  # Filter by agency
    "conditions[type][]": "NOTICE",
    "conditions[term]": "Land Acquisition",  # Search term
    "fields[]": "raw_text_url",
    "order": "relevance",  # Sort order
    "per_page": 2  # Number of results per page
}

# Make the initial API request
response = requests.get(url, params=params)

# Check for errors
if response.status_code == 200:
    data = response.json()
    raw_text_urls = [doc["raw_text_url"] for doc in data.get("results", [])]
    print(f"Found {len(raw_text_urls)} documents.")
    
    # Process each raw text URL
    for raw_text_url in raw_text_urls:
        print(f"Fetching content from: {raw_text_url}")
        raw_response = requests.get(raw_text_url)
        
        if raw_response.status_code == 200:
            # Extract and print the raw text
            raw_text = raw_response.text
            print(f"Content Preview:\n{raw_text[:2000]}...\n")  # Print the first 1000 characters for preview
            print("-" * 80)
        else:
            print(f"Error fetching raw text: {raw_response.status_code}")
else:
    print(f"Error: {response.status_code} - {response.text}")


Found 2 documents.
Fetching content from: https://www.federalregister.gov/documents/full_text/text/2024/01/25/2024-01467.txt
Content Preview:
<html>
<head>
<title>Federal Register, Volume 89 Issue 17 (Thursday, January 25, 2024)</title>
</head>
<body><pre>
[Federal Register Volume 89, Number 17 (Thursday, January 25, 2024)]
[Notices]
[Pages 4984-4986]
From the Federal Register Online via the Government Publishing Office [<a href="http://www.gpo.gov">www.gpo.gov</a>]
[FR Doc No: 2024-01467]


-----------------------------------------------------------------------

DEPARTMENT OF THE INTERIOR

Bureau of Indian Affairs

[245A2100DD/AAKC001030/A0A501010.999900]


Land Acquisitions; Pascua Yaqui Tribe, Eleven Parcels, Pima 
County, Arizona

AGENCY: Bureau of Indian Affairs, Interior.

ACTION: Notice.

-----------------------------------------------------------------------

SUMMARY: The Assistant Secretary--Indian Affairs made a final agency 
determination to acquire in trust 25.56 acres, mor

Load PLSS data for given section

In [3]:
# Load the PLSS shapefile 
shapefile_path = '/Users/mbraaksma/Downloads/BLM_NATL_PLSS.gdb/ilmocplss.gdb'
div_gdf = gpd.read_file(shapefile_path, 
                    layer='PLSSIntersected', 
                    where="STATEABBR = 'MN' AND TWNSHPNO = '108' AND TWNSHPDIR = 'N' AND RANGENO = '015' AND RANGEDIR = 'W' AND FRSTDIVNO = '01'", 
                    engine='pyogrio')

Warning 1: organizePolygons() received a polygon with more than 100 parts.  The processing may be really slow.  You can skip the processing by setting METHOD=SKIP.


Attempt to create polygon

In [4]:
def calculate_new_point(start_point, bearing, distance):
    # Convert bearing to radians
    bearing_rad = np.deg2rad(bearing)
    
    # Distance to radians conversion (earth radius approx 6371000 meters)
    distance_rad = distance / 6371000.0

    lat1 = np.deg2rad(start_point.y)
    lon1 = np.deg2rad(start_point.x)

    lat2 = np.arcsin(np.sin(lat1) * np.cos(distance_rad) +
                     np.cos(lat1) * np.sin(distance_rad) * np.cos(bearing_rad))

    lon2 = lon1 + np.arctan2(np.sin(bearing_rad) * np.sin(distance_rad) * np.cos(lat1),
                             np.cos(distance_rad) - np.sin(lat1) * np.sin(lat2))

    return Point(np.rad2deg(lon2), np.rad2deg(lat2))

def create_curve(start_point, radius, central_angle, direction='concave southwest'):
    # Placeholder function for curve handling, to be developed as needed
    # This will require more advanced geometric calculations or interpolation
    # Return a list of points along the curve
    return [start_point]  # Dummy placeholder

# Get the northwest corner of the section
section_polygon = div_gdf.geometry.unary_union
northwest_corner = Point(section_polygon.bounds[0], section_polygon.bounds[3])

# Trace the described polygon
points = [northwest_corner]
# South 00 degrees 54 minutes 41 seconds East, 778.98 feet
points.append(calculate_new_point(points[-1], 180 + 0.9114, 778.98 * 0.3048))
# South 44 degrees 55 minutes 49 seconds East, 764.84 feet
points.append(calculate_new_point(points[-1], 180 + 44.9303, 764.84 * 0.3048))
# Start of new line, another point at South 44 degrees 55 minutes 49 seconds East, 5121.99 feet
points.append(calculate_new_point(points[-1], 180 + 44.9303, 5121.99 * 0.3048))
# Curve segment with radius 1083.65 feet and central angle 20 degrees 34 minutes 11 seconds
curve_points = create_curve(points[-1], 1083.65 * 0.3048, 20 + 34/60 + 11/3600)
points.extend(curve_points)

# Close the polygon if needed
# points.append(points[0])

# Create a polygon from points
polygon = Polygon(points)

# Visualize or save the new polygon
gdf_polygon = gpd.GeoDataFrame(geometry=[polygon], crs=div_gdf.crs)


Plot

In [5]:
# Plot
m = div_gdf.drop(columns=['SOURCEDATE', 'REVISEDDATE']).explore(name="Section")
m = gdf_polygon.explore(m=m, color="red", name="Parcel")
# this is completely optional
folium.LayerControl().add_to(m)

m

In [10]:
import re

def parse_bearings_distances(text):
    """Parse bearings and distances from the descriptive text."""
    # Regular expression patterns
    bearing_pattern = r'(North|South) (\d+) degrees (\d+) minutes (\d+) seconds (East|West)'
    distance_pattern = r'(\d+\.\d+) feet'

    # Find all bearings and distances
    bearings = re.findall(bearing_pattern, text)
    distances = re.findall(distance_pattern, text)

    # Convert bearings to decimal degrees
    def convert_bearing(match):
        direction1, degrees, minutes, seconds, direction2 = match
        degrees = int(degrees)
        minutes = int(minutes)
        seconds = int(seconds)
        decimal = degrees + (minutes / 60.0) + (seconds / 3600.0)
        # Adjust for cardinal directions
        if direction1 == 'South':
            decimal = -decimal
        if direction2 == 'West':
            decimal = -decimal
        return decimal

    decimal_bearings = [convert_bearing(b) for b in bearings]
    distances = [float(d) for d in distances]

    return decimal_bearings, distances

# Example text
text = """Commencing at the northwest corner of said Section 1; thence on an assumed bearing of 
South 00 degrees 54 minutes 41 seconds East along the west line of said Section 1 a distance 
of 778.98 feet; thence South 44 degrees 55 minutes 49 seconds East 764.84 feet to the north 
line of said Southwest Quarter of the Northwest Quarter to the point of beginning of the line 
to be described; thence South 44 degrees 55 minutes 49 seconds East 5121.99 feet; thence 
southeasterly 389.04 feet to the south line of said Section 1 along a tangential curve concave 
to the southwest having a radius of 1083.65 feet and a central angle of 20 degrees 34 minutes 
11 seconds and there terminating."""

# Parse the text
bearings, distances = parse_bearings_distances(text)
print("Bearings:", bearings)
print("Distances:", distances)


Bearings: [-0.9113888888888889, -44.930277777777775, -44.930277777777775]
Distances: [778.98, 764.84, 5121.99, 389.04, 1083.65]


In [6]:
import math
import numpy as np
from shapely.geometry import Polygon

def create_polygon_from_bearings(start_point, bearings, distances):
    """
    Creates a polygon from bearings and distances.

    Args:
        start_point (tuple): The starting point (x, y) of the polygon.
        bearings (list): List of bearings in degrees.
        distances (list): List of distances corresponding to the bearings.

    Returns:
        shapely.geometry.Polygon: The created polygon.
    """

    x, y = start_point
    points = [start_point]

    for bearing, distance in zip(bearings, distances):
        # Convert bearing to radians
        bearing_rad = math.radians(bearing)

        # Calculate the next point's coordinates
        x += distance * math.sin(bearing_rad)
        y += distance * math.cos(bearing_rad)

        points.append((x, y))

    # Close the polygon
    points.append(start_point)

    return Polygon(points)

# Example usage
start_point = (0, 0)
bearings = [0, 90, 180, 270]  # North, East, South, West
distances = [100, 100, 100, 100]

polygon = create_polygon_from_bearings(start_point, bearings, distances)

print(polygon)

POLYGON ((0 0, 0 100, 100 100, 100.00000000000001 0, 0.0000000000000142 -0.0000000000000184, 0 0))
